In [10]:
from multiprocessing import Pool

# import Sklearn packages
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold 

# import helper packages
from statistics import mean
import pandas as pd
import matplotlib.pyplot as plt
from numpy import sqrt


# measuring time
import time
start_time = time.time()


# Our constants
lambdas = [0.1,1,10,100,200]
folds = 10
M = 10**3

# Read and split data
df = pd.read_csv("train.csv")

# Initialize root mean squared list (over lambdas)
final_rmses = []
# Iterate over lambdas


def meaning(lam):
    # Initialize root mean squared list (over times)
    main_rmses = []
    # Iterate over M times
    for _ in range(M):
        kf = KFold(n_splits=folds, shuffle=True)
        # Initialize root mean squared list (over folds)
        sub_rmses = []
        # Iterate over folds
        for train_index, test_index in kf.split(df):
            X_train, X_test = df.iloc[train_index,1:], df.iloc[test_index,1:]
            y_train, y_test = df.iloc[train_index,0], df.iloc[test_index,0]

            # Fit and predict Ridge regression
            clf = Ridge(alpha=lam, fit_intercept=False).fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            sub_rmses.append(sqrt(mean_squared_error(y_test, y_pred)))
        main_rmses.append(mean(sub_rmses))
    final_rmses.append(mean(main_rmses))


a_pool = Pool()
result = a_pool.map(meaning, lambdas)
#for lam in lambdas:
print(result)
print("--- %s seconds ---" % (time.time() - start_time))


# Plot mean root meaned errors over lambda
plt.plot(lambdas,final_rmses)
plt.show()

# Export results to output.csv
df_export = pd.DataFrame(final_rmses)
df_export.to_csv('output_intercept1_e3.csv', index=False, header=False)  


KeyboardInterrupt: 